In [2]:
pip install requests pyodbc

Note: you may need to restart the kernel to use updated packages.


In [8]:
import requests
import pyodbc

def fetch_api_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from API: {e}")
        return None

def insert_data_to_sql(data, conn):
    try:
        cursor = conn.cursor()
        cursor.execute('''
        IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='api_data' and xtype='U')
        CREATE TABLE api_data (
            id INT PRIMARY KEY IDENTITY(1,1),
            name NVARCHAR(255),
            value FLOAT  -- Adjust data type based on your requirements
        )
        ''')

        for item in data:
            cursor.execute('''
            INSERT INTO api_data (name, value) VALUES (?, ?)
            ''', (item['name'], item['current_price']))  # Adjust key name as per your data structure

        conn.commit()
    except pyodbc.Error as e:
        print(f"Error inserting data into SQL Server: {e}")

def main():
    url = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd'
    data = fetch_api_data(url)
    
    if data:
        try:
            conn = pyodbc.connect(
                'DRIVER={ODBC Driver 17 for SQL Server};'
                'SERVER=localhost;'
                'DATABASE=MyDatabase;'
                'Trusted_Connection=yes;'
            )
            insert_data_to_sql(data, conn)
        except pyodbc.Error as e:
            print(f"Error connecting to SQL Server: {e}")
        finally:
            if 'conn' in locals() and conn:
                conn.close()

if __name__ == "__main__":
    main()


In [ ]:
data = fetch_api_data(url)
print(data)  # Add this line to inspect the structure of data